In [15]:
from pyspark.sql import SparkSession

In [16]:
spark = SparkSession.builder \
    .appName("02") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 00:16:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/08 00:16:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [23]:
from pyspark.sql.functions import col, range

ImportError: cannot import name 'range' from 'pyspark.sql.functions' (/usr/local/lib/python3.8/dist-packages/pyspark/sql/functions.py)

In [22]:
col_sample = col('test')
print(col_sample)
print(type(col_sample))

Column<'test'>
<class 'pyspark.sql.column.Column'>


In [24]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType
from datetime import date

# 스키마 정의
schema = StructType([
    StructField("product", StringType(), True),
    StructField("country", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("date", DateType(), True)
])

# 데이터 생성
data = [
    ("Walnut", "Korea", 12.3, date(2025, 10, 1)),
    ("Walnut", "USA", 10.5, date(2025, 10, 1)),
    ("Almond", "Korea", 9.7, date(2025, 10, 1)),
    ("Almond", "USA", 8.9, date(2025, 10, 1)),
    ("Walnut", "Korea", 12.6, date(2025, 10, 8)),
    ("Walnut", "USA", 10.8, date(2025, 10, 8))
]

# DataFrame 생성
df = spark.createDataFrame(data, schema)

In [32]:
from pyspark.sql.functions import expr, concat, lit

In [41]:
# pythonic 
df.select(concat(col('price'), lit(' USD')).alias('price_usd')).show()

+---------+
|price_usd|
+---------+
| 12.3 USD|
| 10.5 USD|
|  9.7 USD|
|  8.9 USD|
| 12.6 USD|
| 10.8 USD|
+---------+



In [40]:
# sql 스러움
df.select(expr("concat(price, ' USD') as price_usd")).show()

+---------+
|price_usd|
+---------+
| 12.3 USD|
| 10.5 USD|
|  9.7 USD|
|  8.9 USD|
| 12.6 USD|
| 10.8 USD|
+---------+



In [42]:
df.createOrReplaceTempView('market_price')

In [44]:
spark.sql('select * from market_price').show()

+-------+-------+-----+----------+
|product|country|price|      date|
+-------+-------+-----+----------+
| Walnut|  Korea| 12.3|2025-10-01|
| Walnut|    USA| 10.5|2025-10-01|
| Almond|  Korea|  9.7|2025-10-01|
| Almond|    USA|  8.9|2025-10-01|
| Walnut|  Korea| 12.6|2025-10-08|
| Walnut|    USA| 10.8|2025-10-08|
+-------+-------+-----+----------+



In [45]:
df.select('product', 'price').show()

+-------+-----+
|product|price|
+-------+-----+
| Walnut| 12.3|
| Walnut| 10.5|
| Almond|  9.7|
| Almond|  8.9|
| Walnut| 12.6|
| Walnut| 10.8|
+-------+-----+



## expr, selectExpr

In [47]:
df.select(expr('concat(product, " ", price) AS product_price')).show()

+-------------+
|product_price|
+-------------+
|  Walnut 12.3|
|  Walnut 10.5|
|   Almond 9.7|
|   Almond 8.9|
|  Walnut 12.6|
|  Walnut 10.8|
+-------------+



In [48]:
df.selectExpr('concat(product, " ", price) as product_price').show()

+-------------+
|product_price|
+-------------+
|  Walnut 12.3|
|  Walnut 10.5|
|   Almond 9.7|
|   Almond 8.9|
|  Walnut 12.6|
|  Walnut 10.8|
+-------------+



In [50]:
df.select('product', 'price').sort(col('price'), ascending=False).selectExpr('concat(product, " ", price) as product_price').show()

+-------------+
|product_price|
+-------------+
|  Walnut 12.6|
|  Walnut 12.3|
|  Walnut 10.8|
|  Walnut 10.5|
|   Almond 9.7|
|   Almond 8.9|
+-------------+



In [52]:
df.selectExpr('*', 'case when price > 10 then "expensive" else "cheap" end as review').show()

+-------+-------+-----+----------+---------+
|product|country|price|      date|   review|
+-------+-------+-----+----------+---------+
| Walnut|  Korea| 12.3|2025-10-01|expensive|
| Walnut|    USA| 10.5|2025-10-01|expensive|
| Almond|  Korea|  9.7|2025-10-01|    cheap|
| Almond|    USA|  8.9|2025-10-01|    cheap|
| Walnut|  Korea| 12.6|2025-10-08|expensive|
| Walnut|    USA| 10.8|2025-10-08|expensive|
+-------+-------+-----+----------+---------+



### 집계

In [57]:
from pyspark.sql.functions import avg, count

In [73]:
# 단순히 특정 컬럼의 통계치만 볼 때 사용하면 좋을듯
df.select(avg('price')).show()

+------------------+
|        avg(price)|
+------------------+
|10.799999999999999|
+------------------+



In [66]:
df.select(avg('price').alias('price_avg')).groupBy('country').sort('price_avg') # XX 

AttributeError: 'GroupedData' object has no attribute 'sort'

In [71]:
# expr 방식
df.groupBy('country') \
    .agg(expr('avg(price) as price_avg')) \
    .sort('price_avg') \
    .show()

[Stage 33:>                                                         (0 + 2) / 2]

+-------+------------------+
|country|         price_avg|
+-------+------------------+
|    USA|10.066666666666668|
|  Korea|11.533333333333333|
+-------+------------------+



In [67]:
# Dataframe 방식
df.groupBy('country') \
  .agg(avg('price').alias('price_avg')) \
  .sort('price_avg') \
  .show()

+-------+------------------+
|country|         price_avg|
+-------+------------------+
|    USA|10.066666666666668|
|  Korea|11.533333333333333|
+-------+------------------+



### 컬럼 추가 (WITH UDF)

In [78]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def get_currency(country):
    if country == 'Korea':
        return 'KRW'
    elif country == 'USA':
        return 'USD'
    else:
        return None

get_currency_udf = udf(get_currency, StringType())

In [87]:
df.select('*') \
    .withColumn('currency', get_currency_udf('country')) \ # withColumn('컬럼명', 표현식)
    .show()

[Stage 40:>                                                         (0 + 1) / 1]

+-------+-------+-----+----------+--------+
|product|country|price|      date|currency|
+-------+-------+-----+----------+--------+
| Walnut|  Korea| 12.3|2025-10-01|     KRW|
| Walnut|    USA| 10.5|2025-10-01|     USD|
| Almond|  Korea|  9.7|2025-10-01|     KRW|
| Almond|    USA|  8.9|2025-10-01|     USD|
| Walnut|  Korea| 12.6|2025-10-08|     KRW|
| Walnut|    USA| 10.8|2025-10-08|     USD|
+-------+-------+-----+----------+--------+



df.select('*') \
    .withColumn('currency', get_currency_udf('country')).explain()

In [91]:
from pyspark.sql.functions import when

df.select('*') \
    .withColumn("currency",
        when(col("country") == "Korea", "KRW")
        .when(col("country") == "USA", "USD")
        .otherwise(None)
   ) \
    .explain()

== Physical Plan ==
*(1) Project [product#0, country#1, price#2, date#3, CASE WHEN (country#1 = Korea) THEN KRW WHEN (country#1 = USA) THEN USD END AS currency#366]
+- *(1) Scan ExistingRDD[product#0,country#1,price#2,date#3]




- UDF를 쓰면 Spark가 JVM ↔ Python 프로세스 간 데이터 직렬화/역직렬화(Serialize/Deserialize) 를 해야 한다.
- 그래서 물리적 실행 계획에서 BatchEvalPython 단계가 생긴다.
-  ➡️ BatchEvalPython
이게 바로 Python 프로세스 호출 과정이다.
- 즉, Spark가 각 파티션 데이터를 JVM에서 Python으로 넘기고,
Python에서 함수를 실행한 뒤 다시 JVM으로 결과를 돌려보냅니다.
- 이 과정은 병렬 처리 + 네이티브 코드 최적화(WholeStageCodegen) 를 깨트려요 😢
즉, Catalyst 옵티마이저가 인식할 수 없고, 코드 생성(코드제너레이션)도 불가능해집니다.

### column rename

In [92]:
df.withColumnRenamed('date', 'export_date').show()

+-------+-------+-----+-----------+
|product|country|price|export_date|
+-------+-------+-----+-----------+
| Walnut|  Korea| 12.3| 2025-10-01|
| Walnut|    USA| 10.5| 2025-10-01|
| Almond|  Korea|  9.7| 2025-10-01|
| Almond|    USA|  8.9| 2025-10-01|
| Walnut|  Korea| 12.6| 2025-10-08|
| Walnut|    USA| 10.8| 2025-10-08|
+-------+-------+-----+-----------+



## 대소문자 구분

In [94]:
spark.conf.get('spark.sql.caseSensitive')

'false'

In [96]:
spark.conf.set('spark.sql.caseSensitive', True)
spark.conf.get('spark.sql.caseSensitive')

'true'

### casting

In [97]:
from pyspark.sql.functions import cast

In [98]:
df.withColumn('date_string', col('date').cast('string')).show()

+-------+-------+-----+----------+-----------+
|product|country|price|      date|date_string|
+-------+-------+-----+----------+-----------+
| Walnut|  Korea| 12.3|2025-10-01| 2025-10-01|
| Walnut|    USA| 10.5|2025-10-01| 2025-10-01|
| Almond|  Korea|  9.7|2025-10-01| 2025-10-01|
| Almond|    USA|  8.9|2025-10-01| 2025-10-01|
| Walnut|  Korea| 12.6|2025-10-08| 2025-10-08|
| Walnut|    USA| 10.8|2025-10-08| 2025-10-08|
+-------+-------+-----+----------+-----------+



In [102]:
df.withColumn('one', lit(1)) \
    .selectExpr('int(one) + 100') \
    .show()

+-----------+
|(one + 100)|
+-----------+
|        101|
|        101|
|        101|
|        101|
|        101|
|        101|
+-----------+



In [103]:
from pyspark.sql.functions import monotonically_increasing_id

df.select(monotonically_increasing_id()).show()

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                   8589934592|
|                   8589934593|
|                   8589934594|
+-----------------------------+



In [108]:
spark.conf.get('spark.sql.session.timeZone')

'Asia/Seoul'

In [107]:
spark.conf.set('spark.sql.session.timeZone', 'Asia/Seoul')

In [112]:
from pyspark.sql.types import StringType, IntegerType

data = [
    ("Alice", 25, "Korea"),
    ("Bob", None, "USA"),
    ('Cathy', 30, "Japan"),
    ("David", 40, None),
    ('Eve', None, None)
]

schema = StructType([
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), True),
    StructField("country", StringType(), True)
])

person = spark.createDataFrame(data, schema)

In [ ]:
new_person = person.